# **Bootstrapping**

**Author:**
[Anthony Strittmatter](http://www.anthonystrittmatter.com)

In the following, we use a Monte-Carlo simulation to illustrate the use of bootstrapping to calculate standard errors of estimators. Monte-Carlo simulations generate artificial random data. Because the data is artificial, we have control over the data generating process (DGP). Accordingly, we know the true underlying data model. 

## Load Packages

In [2]:
########################  Load Packages  ########################

# Load packages
library(MASS)

print('All packages successfully installed and loaded.')

[1] "All packages successfully installed and loaded."


## Data Generating Process (DGP)

### Select Parameters for DGP

In [3]:
########################  Tuning Parameter Selection  ########################

N = 1000 # Select sample size
replications = 1000 # Select number of replications
mu <- cbind(0,0) # Select expected value of potential outcomes; 
# here expected value of both potential outcomes is zero, such that the true effect is zero
vcm = diag(2) # Select variance covariance matrix; here just identity matrix

print('Tuning parameters selected.')

[1] "Tuning parameters selected."


### Generate Data

We simulate the two potential outcomes $Y^1$ and $Y^0$ from a multivariate normal distribution. We indicate the binary treatment variable by $D$. For simplicity, we simulate random treatment assigment, such that no additional control variables are required to identify the ATE. Furthermore, we fix the share of treated to be exactly 50%. We generate the observed outcome using the observational rule $Y = D\cdot Y^1 + (1-D) \cdot Y^0$. In reality, we would only observe $(Y,D)$ and *not* $(Y^1,Y^0)$.

In [4]:
########################  Generate the Simulated Data  ########################

set.seed(123456789) # Set starting value for random number generator

# Potential outcomes
potential_outcomes <- mvrnorm(n = N, mu, vcm)
colnames(potential_outcomes) <-  c("Y1","Y0")
# The potential outcomes would be (partly) unobserved in reality

# Generate treatment dummy
u <- rnorm(n=N)
D <- matrix(0, nrow=N, ncol =1)
D[u>median(u),] <-1
# We make sure that exactly 50% of the sample are treated 

# Observed outcome
Y <- D*potential_outcomes[,1] + (1-D)*potential_outcomes[,2] # Observational rule

# We would observe the following data in reality
observed_data <- cbind(Y,D)
colnames(observed_data) <-  c("Y","D")

print('Data is generated.')

[1] "Data is generated."


## True Average Treatment Effects (ATE)

We know the true ATE and true standard errors of the ATE from the DGP. In reality, they would not be observed. We can only observe them because we simulated the data.

In [5]:
########################  True Effects  ########################

print(paste0("True ATE: ", round(mu[,1] - mu[,2], digits=3)))
print(paste0("True Standard Error: ", round(sqrt((vcm[1,1] + vcm[2,2]))/sqrt(sum(D)), digits=3)))

[1] "True ATE: 0"
[1] "True Standard Error: 0.063"


## Estimated Effects

We can use the naive estimater for ATE, because the treatment $D$ is randomised in this simulation.

In [6]:
########################  Estimated ATE  ########################

delta <- mean(Y[D==1]) - mean(Y[D==0]) # naive estimator

print(paste0("True ATE: ", round(mu[,1] - mu[,2], digits=3)))
print(paste0("Estimated ATE: ", round(delta, digits=3)))

[1] "True ATE: 0"
[1] "Estimated ATE: 0.013"


## Bootstrapping

Now we use a non-parametric bootstrapping aproach to calculate the standard errors of the estimated ATE. First, we have to draw bootstrap samples of size $N$ with replacement. For this purpose, we can use the *sample()* function. Let us test this function with $N=10$.

In [7]:
set.seed(123456789) # Set starting value for random number generator
sample(10, replace=T) # Generate random bootstrap sample with replacement of size 10

[1]  4  2  7  6  7  3  9 10 10  5

Each number represents an observation row in the data. You see that some observations are drawn several times and other observations are not drawn at all. For example, we observe two times observation 7 in the bootstrap sample, but observation 1 does not appear at all. In each draw, each observation hs the same probability to be drawn. The bootstrap sample should have the same sample size as the original sample ($N=1000$). 

Second, we have to estimate the ATE in the bootstrap sample. Then we draw a new random bootstrap sample and estimate the ATE again. To repeat these step many times, we can use a *for*-loop.

In [8]:
########################  Bootstrapping  ########################

set.seed(123456789) # Set starting value for random number generator
boot <- matrix(NA, nrow=replications, ncol=1) # Define matrix to store ATEs

for (rep in c(1:replications)) { # for-loop: cycles through all B replications (here 1000) 
 
    c <- sample(N, replace=T) # Generate random bootstrap sample with replacement of size N
    boot[rep,] <- sum(Y[c]*D[c])/sum(D[c]) - sum(Y[c]*(1-D[c]))/sum(1-D[c]) # Calculate ATE using naive estimator
    # I calculate the conditional average manually to make sure that the right observations are used 
    
}

print(paste0("True Standard Error: ", round(sqrt((vcm[1,1] + vcm[2,2]))/sqrt(sum(D)), digits=3)))
print(paste0("Estimated Standard Error: ", round(sqrt(var(boot)), digits=3)))

[1] "True Standard Error: 0.063"
[1] "Estimated Standard Error: 0.066"


Note that in this simple example the OLS estimator provides equivalent results.

In [9]:
########################  OLS  ########################

ols <- lm(Y~D, data = as.data.frame(observed_data))
summary(ols)


Call:
lm(formula = Y ~ D, data = as.data.frame(observed_data))

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6321 -0.6966 -0.0064  0.7160  3.4792 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept) -0.04013    0.04622  -0.868    0.385
D            0.01310    0.06536   0.200    0.841

Residual standard error: 1.033 on 998 degrees of freedom
Multiple R-squared:  4.027e-05,	Adjusted R-squared:  -0.0009617 
F-statistic: 0.0402 on 1 and 998 DF,  p-value: 0.8411
